Código do webscrapping realizado 

In [ ]:
from bs4 import BeautifulSoup
from requests import get
from time import sleep
from random import randint
from warnings import warn
from time import time
from IPython.core.display import clear_output
import re

In [42]:
pages = [str(i) for i in range(57,89)]

start_time = time()
requests = 0
    # Lists to store the scraped data in
tipos = []
dorms = []
baths = []
parks = []
bairros = []
prices = []
suites = []
mobilias = []	
# For every page in the interval 
for page in pages:
        
    # Make a get request
    response = get('https://www.cardinali.com.br/pesquisa-de-imoveis/?locacao_venda=L&id_cidade[]=190&id_tipo_imovel[]=164&id_tipo_imovel[]=1&id_tipo_imovel[]=3&id_tipo_imovel[]=55&id_tipo_imovel[]=37&id_tipo_imovel[]=4&id_tipo_imovel[]=6&id_tipo_imovel[]=61&id_tipo_imovel[]=8&id_tipo_imovel[]=161&id_tipo_imovel[]=10&id_tipo_imovel[]=40&id_tipo_imovel[]=12&id_tipo_imovel[]=13&finalidade=0&dormitorio=0&garagem=0&vmi=&vma=&&pag=' + page)
    # Pause the loop
    sleep(randint(15,25))
        
    # Monitor the requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    #clear_output(wait = True)
              
    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
              
    # Break the loop if the number of requests is greater than expected
    if requests > 99:
        warn('Number of requests was greater than expected.')  
        break 
        
    # Parse the content of the request with BeautifulSoup
    page_html = BeautifulSoup(response.text, 'html.parser')
    rent_containers = page_html.find_all('div', class_ = 'item col-sm-6 col-md-4 col-lg-3')
    # Extract data from individual rent container
    for container in rent_containers:
    # type
        tipo = container.h3.a.text
        tipos.append(tipo)
        
    # dorms
        dorm = int(container.ul.findChildren()[0].text)
        dorms.append(dorm)
        
    # baths
        bath = container.ul.findChildren()[2].text
        if(bool(re.findall('�',bath))==True):
            bath = 0
        else:
            bath = int(container.ul.findChildren()[2].text)
        baths.append(bath)
    
    # parking
        if len(container.ul.findChildren())>4:
            park = container.ul.findChildren()[4].text
            if(bool(re.findall('�',park))==True):
                park = 0
            else:
                park = int(container.ul.findChildren()[4].text)
        else:
            park = 0
        parks.append(park)
        
    # bairro
        bairro = container.h3.small.text.replace('Bairro', '').replace(':', '')
        bairros.append(bairro)
    
    #price
        price = 'NaN'
        price_containers = container.find(class_ = 'price_lv')
        if price_containers is not None:
            price = float(price_containers.get_text(strip=True).replace('.','').replace(',','.'))
        prices.append(price)

        #webscraping each target page
        link = container.a['href']
        # Send an HTTP request to the target page
        target_response = get('https://www.cardinali.com.br/alugar/'+ link)

        # Parse the HTML content of the target page
        target_soup = BeautifulSoup(target_response.content, 'html.parser')

        # Extract the desired data from the target page
        target_suite = target_soup.find_all('div', {'class': 'text-center'})
        suite = 0
        for i in range(0, len(target_suite)):
            target = target_suite[i].text
            if 'Suítes' in target or 'Suíte' in target:
                suite = int(target.split()[1])
        suites.append(suite)

        target_mobiliado = target_soup.find_all('li', {'class': 'enabled col-sm-4'})
        mobiliado = 'Não'
        for i in range(len(target_mobiliado)):
                target3 = target_mobiliado[i].text
                if 'Mobiliado' in target3 or 'Armários' in target3:
                    mobiliado = 'Sim'
        mobilias.append(mobiliado)

Request:1; Frequency: 0.03605415171460481 requests/s
Request:2; Frequency: 0.025977595223422002 requests/s
Request:3; Frequency: 0.022007533901581174 requests/s
Request:4; Frequency: 0.022705362409073195 requests/s
Request:5; Frequency: 0.022178872783711372 requests/s
Request:6; Frequency: 0.021006983008388887 requests/s
Request:7; Frequency: 0.02027669368555399 requests/s
Request:8; Frequency: 0.020775118650348673 requests/s
Request:9; Frequency: 0.02138376632127138 requests/s
Request:10; Frequency: 0.021686991211342137 requests/s
Request:11; Frequency: 0.022012818698217612 requests/s
Request:12; Frequency: 0.022121379888129556 requests/s
Request:13; Frequency: 0.022303797779708194 requests/s
Request:14; Frequency: 0.021902332695647883 requests/s
Request:15; Frequency: 0.021862324944830838 requests/s
Request:16; Frequency: 0.02187610915213719 requests/s
Request:17; Frequency: 0.02135331992261474 requests/s
Request:18; Frequency: 0.020658640737093045 requests/s
Request:19; Frequency: 0

In [43]:
len(tipos)

512

In [44]:
import pandas as pd
teste_df = pd.DataFrame({'Tipo': tipos,
                       'Dormitórios': dorms,
                       'Banheiros': baths,
                       'Garagens': parks,
                       'Bairro': bairros,
                       'Valor': prices,
                       'Suítes': suites,
                       'Mobiliado': mobilias})
print(teste_df.info())
teste_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Tipo         512 non-null    object 
 1   Dormitórios  512 non-null    int64  
 2   Banheiros    512 non-null    int64  
 3   Garagens     512 non-null    int64  
 4   Bairro       512 non-null    object 
 5   Valor        512 non-null    float64
 6   Suítes       512 non-null    int64  
 7   Mobiliado    512 non-null    object 
dtypes: float64(1), int64(4), object(3)
memory usage: 32.1+ KB
None


,Tipo,Dormitórios,Banheiros,Garagens,Bairro,Valor,Suítes,Mobiliado
0,Apartamento Padrão,2,1,1,Recreio São Judas Tadeu,1042.50,0,Sim
1,Apartamento Padrão,2,1,1,Vila Izabel,1625.00,0,Sim
2,Apartamento Padrão,2,2,0,Vila Jacobucci,556.25,0,Sim
3,Apartamento Padrão,2,1,2,Jardim Alvorada,1625.00,2,Sim
4,Casa Padrão,3,1,2,Jardim Nova Santa Paula,2375.00,1,Sim


In [45]:
teste_df.to_csv('tabela_completa3.csv')

In [46]:
import os
import glob
import pandas as pd
#list all csv files only
csv_files = glob.glob('*.{}'.format('csv'))
csv_files

['tabela_completa1.csv', 'tabela_completa2.csv', 'tabela_completa3.csv']

In [47]:
df_concat = pd.concat([pd.read_csv(f) for f in csv_files ], ignore_index=True)
df_concat

,Unnamed: 0,Tipo,Dormitórios,Banheiros,Garagens,Bairro,Valor,Suítes,Mobiliado
0,0,Apartamento Padrão,1,1,2,Jardim Paraíso,1750.00,0,Sim
1,1,Casa Padrão,3,1,3,Jardim Embaré,2083.75,1,Não
2,2,Apartamento Padrão,2,1,1,Vila Celina,1667.50,0,Não
3,3,Casa Padrão,1,1,1,Jardim Social Presidente Collor,695.00,0,Não
4,4,Casa Padrão,3,1,1,Vila Faria,2778.75,1,Sim
...,...,...,...,...,...,...,...,...,...
1387,507,Apartamento Padrão,2,2,1,Vila Monteiro Gleba I,2062.50,0,Sim
1388,508,Casa Padrão,3,1,1,Jardim Cruzeiro do Sul,2050.00,1,Sim
1389,509,Casa Padrão,4,3,1,Centro,3750.00,0,Sim
1390,510,Apartamento Padrão,1,1,1,Cidade Jardim,1000.00,0,Sim


In [48]:
df_concat.drop(['Unnamed: 0'], axis=1, inplace=True)

In [24]:
#df_concat.drop(['Unnamed: 0.1'], axis=1, inplace=True)

In [49]:
df_concat

,Tipo,Dormitórios,Banheiros,Garagens,Bairro,Valor,Suítes,Mobiliado
0,Apartamento Padrão,1,1,2,Jardim Paraíso,1750.00,0,Sim
1,Casa Padrão,3,1,3,Jardim Embaré,2083.75,1,Não
2,Apartamento Padrão,2,1,1,Vila Celina,1667.50,0,Não
3,Casa Padrão,1,1,1,Jardim Social Presidente Collor,695.00,0,Não
4,Casa Padrão,3,1,1,Vila Faria,2778.75,1,Sim
...,...,...,...,...,...,...,...,...
1387,Apartamento Padrão,2,2,1,Vila Monteiro Gleba I,2062.50,0,Sim
1388,Casa Padrão,3,1,1,Jardim Cruzeiro do Sul,2050.00,1,Sim
1389,Casa Padrão,4,3,1,Centro,3750.00,0,Sim
1390,Apartamento Padrão,1,1,1,Cidade Jardim,1000.00,0,Sim


In [52]:
df_concat.duplicated().sum()

224

In [53]:
df_concat.shape

(1392, 8)

In [54]:
df_concat.to_csv('tabela_aluguel_completa.csv')